2012- Robert M Dorazio & J. Andrew Royle. Estimating Size and Composition of Biological Communities by Modeling the occurrence of Species.
https://doi.org/10.1198/016214505000000015

Zero-inflated model

MacKenzie et al. 2006

MacKenzie et al. 2002

In [ ]:
import shutil
import os
import zipfile
from joblib import dump, load
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.patches import Patch  # for legend patch
from scipy.optimize import minimize_scalar
from scipy.stats import norm, beta

In [ ]:
# Remove the directory and all its contents
if os.path.exists('MinduPark'):
    shutil.rmtree('MinduPark')
    print("Folder 'MinduPark' removed.")
if os.path.exists('MinduPark_without_sauim'):
    shutil.rmtree('MinduPark_without_sauim')
    print("Folder 'MinduPark_without_sauim' removed.")

Auxiliar funtions

In [ ]:
def w_mle_variance_ci(k, K, p11):
    """
    Estima w_hat, Var(w_hat) e SE(w_hat)
    no modelo q = w * p11, k ~ Bin(K, q).

    Parâmetros:
      k    : nº de sítios com ≥1 detecção
      K    : nº total de sítios
      p11  : prob. de detecção quando ocupado (fixa/conhecida)

    Retorna:
      dict com w_hat, var_wald, se_wald
    """
    if not (0 <= k <= K):
        raise ValueError("Exige 0 <= k <= K.")
    if not (0 < p11 <= 1):
        raise ValueError("Exige 0 < p11 <= 1.")

    # MLE fechada
    q_hat = k / K
    w_hat = np.clip(q_hat / p11, 0.0, 1.0)

    # Variância (delta-método) e SE
    var_wald = (q_hat * (1 - q_hat)) / (K * (p11 ** 2))
    se_wald = float(np.sqrt(max(var_wald, 0.0)))

    return {
        "w_hat": float(w_hat),
        "var_wald": float(var_wald),
        "se_wald": se_wald,
    }

## Ocumpacy modeling

Automatically download the necessary files

In [ ]:
!wget https://github.com/juancolonna/Sauim/raw/main/MinduPark.zip -O MinduPark.zip
!wget https://github.com/juancolonna/Sauim/raw/main/MinduPark_without_sauim.zip -O MinduPark_without_sauim.zip
!wget https://github.com/juancolonna/Sauim/raw/main/ocsvm_filtered.joblib -O ocsvm_filtered.joblib

In [ ]:
# Open and extract files
with zipfile.ZipFile('MinduPark.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

with zipfile.ZipFile('MinduPark_without_sauim.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

print(f'Files extracted')

In [ ]:
clf = load("ocsvm_filtered.joblib")

In [ ]:
cm = np.array([[179,0],[4,32]])

# Fixed parameters
p10 = cm[0,1]/cm.sum(axis=1)[0]  # prob. of "false detection"
p11 = cm[1,1]/cm.sum(axis=1)[1]  # prob. of "detection"
p01 = 1-p11
p00 = 1-p10

eps = 1e-10

print(f"p11:{p11}, p10:{p10}, p01:{p01}, p00:{p00}")

Semples monitoring with Sauim occurrences

In [ ]:
y = []
T = []
detections_per_record = []

for i in range(8):
    MinduPark_embedding_vectors = np.load("MinduPark/MinduPark_vectors_filtered_"+str(i)+".npy")
    decision_scores = clf.decision_function(MinduPark_embedding_vectors)
    MinduPark_preds = np.where(decision_scores >= 0, 1, 0)
    y.append(np.sum(MinduPark_preds == 1))
    T.append(len(MinduPark_preds))
    detections_per_record.append(MinduPark_preds)

y = np.array(y)
T = np.array(T)
print(y) # How many detections per record
print(T) # How many segments per record

In [ ]:
detections = int(np.sum(y > 0))
detections

In [ ]:
from scipy.stats import norm

K = len(y)
mle_w = []
logLike = []
w_lo = []   # SE-based lower band for w
w_hi = []   # SE-based upper band for w

eps = 1e-12

for k in range(len(y)+1):
    def loglike(w):
        if w <= 0 or w >= 1:
            return -np.inf
        q = np.clip(w * p11, eps, 1 - eps)
        return k*np.log(q) + (K - k)*np.log(1 - q)

    res = minimize_scalar(lambda w: -loglike(w), bounds=(0,1), method="bounded")
    w_hat_k = res.x
    mle_w.append(w_hat_k)
    logLike.append(-res.fun)

    # ---- use SE_wald directly ----
    stats_k = w_mle_variance_ci(k, K, p11)
    se_k = stats_k["se_wald"]

    lo = np.clip(w_hat_k - se_k, 0.0, 1.0)
    hi = np.clip(w_hat_k + se_k, 0.0, 1.0)

    w_lo.append(lo)
    w_hi.append(hi)

detections = int(np.sum(y > 0))

# --- plotting (unchanged except labels) ---
fig, ax1 = plt.subplots()
x = np.arange(len(mle_w))

l1, = ax1.plot(x, mle_w, label="$\\psi$ (MLE)")
ax1.set_ylabel("$\\psi$ (MLE)", color="blue", fontsize=14)
ax1.set_ylim(-0.01, 1.01)

band = ax1.fill_between(x, w_lo, w_hi, alpha=0.2, color=l1.get_color())
se_patch = Patch(facecolor=l1.get_color(), alpha=0.2, label=f"±SE")

y_hat_at_det = mle_w[detections]
lo_at_det = w_lo[detections]
hi_at_det = w_hi[detections]
yerr_det = np.array([[y_hat_at_det - lo_at_det], [hi_at_det - y_hat_at_det]])
eb = ax1.errorbar([detections], [y_hat_at_det], yerr=yerr_det, fmt='o', color='black',
                  capsize=3, label="detections")

ax2 = ax1.twinx()
l2, = ax2.plot(x, logLike, label="log-like", color="red", linestyle="--")
ax2.set_ylabel("log-likelihood", color="red", fontsize=14)

ax1.set_xlabel("Number of detections", fontsize=14)
lines = [l1, se_patch, l2, eb]
labels = [ln.get_label() for ln in lines]
ax1.legend(lines, labels, loc="lower right")

# tamanho dos números dos eixos (tiques)
ax1.tick_params(axis='both', labelsize=14)  # x e y do ax1
ax2.tick_params(axis='y', labelsize=14)     # y do ax2 (direita)

fig.tight_layout()
ax1.grid(True, axis='y', which='both')
ax2.grid(False)
ax2.patch.set_visible(False)
ax1.set_axisbelow(True)

plt.savefig("likelihood.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Dados agregados:
k = int(np.sum(y > 0))   # nº de sítios com ≥1 detecção
K = int(len(y))          # nº total de sítios
eps = 1e-12              # p/ evitar log(0)

def loglike(w):
    if w <= 0 or w >= 1:
        return -np.inf  # fora do domínio
    q = w * p11
    q = np.clip(q, eps, 1 - eps)
    return k*np.log(q) + (K - k)*np.log(1 - q)

# Minimiza o negativo do log-likelihood
res = minimize_scalar(lambda w: -loglike(w), bounds=(eps, 1 - eps), method="bounded")

print("MLE estimate of w:", res.x)
print("Max log-likelihood:", -res.fun)


In [ ]:
# --- Exemplo de uso ---
# Suponha que você já tenha:
k = int(np.sum(y > 0))
K = int(len(y))

res = w_mle_variance_ci(k, K, p11)
print("ŵ:", res["w_hat"])
print("Var (Wald):", res["var_wald"])
print("SE (Wald):", res["se_wald"])

Samples of monitoring without sauim occurrences

In [ ]:
y = []
T = []
detections_per_record = []

for i in range(8):
    MinduPark_embedding_vectors = np.load("MinduPark_without_sauim/MinduPark_vectors_filtered_"+str(i)+".npy")
    decision_scores = clf.decision_function(MinduPark_embedding_vectors)
    MinduPark_preds = np.where(decision_scores >= 0, 1, 0)
    y.append(np.sum(MinduPark_preds == 1))
    T.append(len(MinduPark_preds))
    detections_per_record.append(MinduPark_preds)

y = np.array(y)
T = np.array(T)
print(y) # How many detections per record
print(T) # How many segments per record

In [ ]:
# --- Exemplo de uso ---
k = int(np.sum(y > 0))
K = int(len(y))

res = w_mle_variance_ci(k, K, p11)
print("ŵ:", res["w_hat"])
print("Var (Wald):", res["var_wald"])
print("SE (Wald):", res["se_wald"])